## 1D PIC (evolution_version) 

Here we compute the 1D version of PIC. The distribution we use corresponds to a particle current going in both directions with the same mean density and opposite velocities. This configuration generates an instibility and we observe it. 

The Electric field is computed from Maxwell's equations (the time derivative of E is used), starting with an initial data for E using $\nabla \cdot \vec{E} = \rho$ (computing the potential and taking the derivative). 

In [243]:
using DifferentialEquations
using Plots
using Statistics
using FFTW
FFTW.set_provider!("mkl")
#import Pkg; Pkg.add("Distributions")
using Distributions


Helper functions are defined in this file.

In [244]:
include("aux_functions.jl")

get_index_and_distance (generic function with 1 method)

### Parameters

    L;            // Domain of solution 0 <= x <= L (in Debye lengths)
    N;            // Number of electrons
    J;            // Number of grid points
    vb;    // Beam velocity
    dt;    // Time-step (in inverse plasma frequencies)
    t_f;  // Simulation run from t = 0. to t = t_f


In [245]:

L = 100
#N = 200000
#N = 20000
#J = 2048
#J = 1024
J = 512
vb = 3
dt = 0.1
t_f = 20.0
t = 0.0
M = convert(Int64,t_f/dt)
#M=1
κ = 2π/L
dx = L/J
x = [dx*(i-1) for i in 1:J] ;
p = (L, N, J, κ, dx)

(100, 200000, 512, 0.06283185307179587, 0.1953125)

### Produce the intial data

The 1D distribution for $\frac{m}{kT} = 1$ is just the normal distribution centered at both $\pm v_p$. With $\sigma = 1$.

**Important** the velocity distribution must be symmetrical, otherwise the resulting $v_{total}$ and $S_{total}$ are no null and we get
different solutions. I think the one that uses the potential is the incorrect one.... because assumes $S_{total} =0$ to compute the potential and then it does not add the constant term needed from the time evolution of $\vec{E}$.


In [246]:
E = zeros(J)
ϕ = zeros(J)
n = zeros(J) #charge density
S = zeros(J) #carge current
du = zeros(2*N+J); # contains r, v and E

r = L*rand(N);
vp = rand(Normal(vb,1),N÷2);
get_density!(u, n, p)
n0 = N/L
get_ϕ!(ϕ, -n/n0 .+ 1., κ) # chenge the sign here to make it consistent with charge conservation and the time derivative of E
get_E_from_ϕ!(ϕ,E,dx)

u = [r; -vp; vp; E];

BoundsError: BoundsError: attempt to access 80512-element Vector{Float64} at index [1:200000]

In [247]:


get_current!(u, S, p)

Coordinate_test(r,L)

println("n_total = $(sum(n/n0 .- 1))")
println("v_total = $(sum([-vp;vp]))")
println("E_total = $(sum(E))")

println("S_total = $(sum(S))")


BoundsError: BoundsError: attempt to access 80512-element Vector{Float64} at index [1:200000]

In [248]:
plt = scatter(u[1:N],u[N+1:2N]
, thickness_scaling = 0.3
#, markersize = 0.3
, title = "phase-space", legend =:false)
#png("t00_1024_2_4")

BoundsError: BoundsError: attempt to access 80512-element Vector{Float64} at index [1:200000]

Initialize some of the intermediate fields we need.

Compute the initial density, potential and electric field.

In [249]:
#plot(x,n/n0 .- 1)
plot(x,n/n0, title = "density", legend = :false)

Find the correspondig potential

In [ ]:
plot(x,ϕ, title = "potential", legend = :false)

In [ ]:
plot(x,E, title = "Electric Field", legend = :false)

Check the histograms for the initial positions and velocities.

In [ ]:
histogram(u[1:N], title = "density", legend = :false)

In [ ]:
histogram(u[N+1:2*N], title = "velocity", legend = :false)

In [ ]:
get_current!(u, S, p)
histogram(S, title = "current", legend = :false)

Now we make the time evolution:

In [ ]:
p_RHSC = N, J, L, dx, n, S, du, get_density!, get_current! ;

for k in 1:M
  RK4_Step!(RHSC,u,t,dt,p_RHSC)
  u = [make_periodic!(u[1:N],L); u[N+1:end]]
  if k % (M÷10) == 0
    #scatter(plt, u[1:N], u[N+1:2*N])
  end
  t = t + dt
end

Plot the final values

The density:

In [ ]:
get_density!(u,n,p)
plot(x,n*L/N .- 1, title = "density", legend =:false)

In [ ]:
histogram(u[1:N], title = "density", legend =:false)

In [ ]:
histogram(u[N+1:2N], title = "velocity", legend =:false)

In [ ]:
scatter(u[1:N],u[N+1:2N]
#, thickness_scaling = 0.3
, markersize = 0.3
, title = "phase-space", legend =:false)
#png("t20_1024_2_4_E")

In [ ]:
plot(x,u[2N+1:end], title="Electric Field")

In [ ]:
E_F = zeros(J)
ϕ_F = zeros(J)
n_F = zeros(J)
get_density!(u, n_F, p)
n0 = N/L
get_ϕ!(ϕ_F, n_F/n0 .- 1., κ)
get_E_from_ϕ!(ϕ_F,E_F,dx)
plot(x,E_F)

In [ ]:
println("averaged total E field = $(sum(E_F))")
println("E_total = $(sum(u[2N+1:end]))")
println("Total velocity = $(sum(u[N+1:2N]))")
println("Total Charge = $(sum(n_F/n0 .- 1.))")